## Solve Model balancing problem with kinetic parameters as variables

In [1]:
import numpy as np
from equilibrator_api import Q_, R, default_T
from model_balancing import _ln_kcatr, _ln_conc_enz, _driving_forces, solve
import cvxpy as cp

In [2]:
args = {}

# Network topology

args["S"] = np.matrix(
    [
        [-1, 0, 0],
        [1, -1, -1],
        [0, 1, 1]
    ], dtype=float
)
args["A_act"] = np.matrix(
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ], dtype=float
)
args["A_inh"] = np.matrix(
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ], dtype=float
)

In [3]:
# Priors for Kinetic and Thermodynamic parameters

args["Keq_gmean"] = Q_([5, 3, 5], "")  # geometric mean (assuming a standard concentration of 1M)
args["Keq_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
)  # log-scale covariance

args["kcatf_gmean"] = Q_([5.0, 5.0, 5.0], "1/s")  # geometric mean
args["kcatf_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

args["kcatr_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

# the K parameters are only for existing connections in the S, A_act and A_inh matrices.
# the order of values is "metabolite-first".
args["Km_gmean"] = Q_([1e-2, 1e-4, 1e-4, 1e-3, 1e-1, 1e-1], "M")
args["Km_ln_cov"] = np.array(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ]
) # log-scale covariance

In [4]:
args["Ka_gmean"] = Q_([], "M")
args["Ka_ln_cov"] = np.array([])
args["Ki_gmean"] = Q_([], "M")
args["Ki_ln_cov"] = np.array([])

In [5]:
# condition dependent data (columns represent conditions)

args["fluxes"] = Q_(
    [
        [2.0, 1.5, 2.5, 2.5],
        [1.0, 1.0, 2.0, 0.5],
        [1.0, 0.5, 0.5, 2.0],
    ], "mM/s"
)

args["conc_enz_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

args["conc_met_gmean"] = Q_(
    [
        [2e-3, 1e-2, 2e-3, 1e-3],
        [2e-4, 1e-4, 3e-4, 1e-4],
        [1e-5, 1e-5, 1e-5, 1e-5],
    ], "M"
)  # geometric mean

args["conc_met_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

In [6]:
fluxes, S, A_act, A_inh = args["fluxes"], args["S"], args["A_act"], args["A_inh"]
ln_Keq = np.log(args["Keq_gmean"].m_as(""))
ln_kcatf = np.log(args["kcatf_gmean"].m_as("1/s"))
ln_Km = np.log(args["Km_gmean"].m_as("M"))
ln_Ka = np.log(args["Ka_gmean"].m_as("M"))
ln_Ki = np.log(args["Ki_gmean"].m_as("M"))
ln_conc_met = np.log(args["conc_met_gmean"].m_as("M"))

args["kcatr_gmean"] = np.exp(_ln_kcatr(ln_kcatf, ln_Km, ln_Keq, S).value) * Q_("1/s")


In [7]:
# check that the driving forces at the mode are all positive
if (_driving_forces(ln_Keq, ln_conc_met, S).value >= 1e-6).all():
    print("Feasible")
else:
    raise Exception("Infeasible")

Feasible


In [8]:
solver = "SCS"
#solver = "ECOS"

for rate_law in ["CM"]:#["S", "1S", "SP", "1SP", "CM"]:
    print(f"************ rate law = {rate_law} **************\n")
    args["conc_enz_gmean"] = np.exp(
        _ln_conc_enz(
            ln_Keq, ln_kcatf, ln_Km, ln_Ka, ln_Ki, ln_conc_met,
            fluxes, S, A_act, A_inh , rate_law=rate_law).value
    ) * Q_("M")
    args["rate_law"] = rate_law

    solve(
        **args,
        solver=solver
    )

************ rate law = CM **************

initial Z-score = 0.00e+00
optimal Z-score = 4.41e-07

Metabolite concentrations (M) =
 [[2.00000109e-03 1.00000394e-02 2.00000469e-03 1.00000000e-03]
 [1.99999974e-04 9.99999847e-05 3.00001397e-04 1.00000073e-04]
 [1.00000072e-05 9.99999296e-06 9.99997419e-06 1.00000050e-05]]

Enzyme concentrations (M) =
 [[0.00652748 0.00090145 0.01081944 0.01070925]
 [0.00030499 0.00041367 0.00053915 0.00020683]
 [0.001212   0.00112234 0.0004362  0.00448935]]

Driving forces (RT) =
 [[3.91229426 6.21488277 3.50682616 3.91229285]
 [4.09445398 3.4013082  4.49992718 3.40130788]
 [4.60526666 3.91212088 5.01073986 3.91212056]]

η(thr) =
 [[0.98 1.   0.97 0.98]
 [0.98 0.97 0.99 0.97]
 [0.99 0.98 0.99 0.98]]

η(kin) =
 [[0.06 0.33 0.05 0.05]
 [0.67 0.5  0.75 0.5 ]
 [0.17 0.09 0.23 0.09]]

η(reg) =
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]




All Z-scores
--------------------------------------------------
enzymes =  0.0
metabolites =  0.0
Keq =  0.0
kcat forw

from model_balancing import _z_score, _ln_capacity, _ln_eta_thermodynamic, _ln_eta_kinetic, _ln_eta_regulation

fluxes, S, A_act, A_inh = args["fluxes"], args["S"], args["A_act"], args["A_inh"]

ln_Keq = cp.Variable(3)
ln_Keq.value = np.log(args["Keq_gmean"].m_as(""))

ln_kcatf = cp.Variable(3, )
ln_kcatf.value = np.log(args["kcatf_gmean"].m_as("1/s"))

ln_Km = cp.Variable(args["Km_gmean"].size)
ln_Km.value = np.log(args["Km_gmean"].m_as("M"))

ln_conc_met = cp.Variable(shape=(3, 4))
ln_conc_met.value = np.log(args["conc_met_gmean"].m_as("M"))

ln_kcatr = _ln_kcatr(ln_kcatf, ln_Km, ln_Keq, S)

driving_forces = _driving_forces(ln_Keq, ln_conc_met, S)

ln_capacity = _ln_capacity(fluxes, ln_kcatf)
ln_eta_thermodynamic = _ln_eta_thermodynamic(driving_forces)
ln_eta_kinetic = _ln_eta_kinetic(ln_conc_met, ln_Km, S, rate_law="S")
ln_eta_regulation = _ln_eta_regulation(ln_conc_met, ln_Ka, ln_Ki, A_act, A_inh)
ln_conc_enz = ln_capacity - ln_eta_thermodynamic - ln_eta_kinetic - ln_eta_regulation

ln_conc_enz.value

z2_scores_met = sum(cp.square((ln_conc_met - np.log(args["conc_met_gmean"].m_as("M"))) / np.log(args["conc_met_gstd"])).flatten())
z2_scores_kcatr = _z_score(ln_kcatr, np.log(args["kcatr_gmean"].m_as("1/s")), args["kcatr_ln_cov"])
z2_scores_Keq = _z_score(ln_Keq, np.log(args["Keq_gmean"].m_as("")), args["Keq_ln_cov"])
z2_scores_kcatf = _z_score(ln_kcatf, np.log(args["kcatf_gmean"].m_as("1/s")), args["kcatf_ln_cov"])
z2_scores_Km = _z_score(ln_Km, np.log(args["Km_gmean"].m_as("M")), args["Km_ln_cov"])
z2_scores_enz = sum(cp.square(
    cp.pos(ln_conc_enz - np.log(args['conc_enz_gmean'].m_as("M"))) / np.log(args["conc_enz_gstd"])
).flatten())

objective = cp.Minimize(
        z2_scores_met +
        z2_scores_Keq +
        z2_scores_kcatf +
        z2_scores_Km +
        z2_scores_kcatr +
        z2_scores_enz
    )

objective.value

_ln_conc_enz(ln_Keq, ln_kcatf, ln_Km, ln_Ka, ln_Ki, ln_conc_met, fluxes, S, A_act, A_inh , rate_law="S").value